In [1]:
import pymysql
import colorExtract

In [2]:
conn = pymysql.connect(host='127.0.0.1', user='ssafy', password='ssafy', db='kkalong', charset='utf8')

In [3]:
cur = conn.cursor()

In [4]:
color = colorExtract.image_preprocess("https://firebasestorage.googleapis.com/v0/b/kkalong-b4cec.appspot.com/o/casual%2Fcasual_0.png?alt=media&token=7fa8ddd5-f8e6-4d0f-9255-72fa037e56bb")

(218, 211, 200)


In [5]:
cur.execute("select * from clothing");
for row in cur:
    img = row[5]
    print(img)
    color = colorExtract.image_preprocess(img)
    print(color)
    cur.execute("insert")

https://firebasestorage.googleapis.com/v0/b/kkalong-b4cec.appspot.com/o/casual%2Fcasual_0.png?alt=media&token=7fa8ddd5-f8e6-4d0f-9255-72fa037e56bb
(218, 211, 200)
라이트그레이
https://firebasestorage.googleapis.com/v0/b/kkalong-b4cec.appspot.com/o/casual%2Fcasual_1.png?alt=media&token=2ae2aaca-a140-4574-b7b1-1b7080e6d8f7
(72, 66, 60)
카키
https://firebasestorage.googleapis.com/v0/b/kkalong-b4cec.appspot.com/o/casual%2Fcasual_2.png?alt=media&token=2ae2aaca-a140-4574-b7b1-1b7080e6d8f7
(169, 152, 113)
회색
https://firebasestorage.googleapis.com/v0/b/kkalong-b4cec.appspot.com/o/casual%2Fcasual_3.png?alt=media&token=2ae2aaca-a140-4574-b7b1-1b7080e6d8f7
(112, 79, 55)
카키
https://firebasestorage.googleapis.com/v0/b/kkalong-b4cec.appspot.com/o/casual%2Fcasual_4.png?alt=media&token=2ae2aaca-a140-4574-b7b1-1b7080e6d8f7
(142, 125, 87)
올리브 그린
https://firebasestorage.googleapis.com/v0/b/kkalong-b4cec.appspot.com/o/casual%2Fcasual_5.png?alt=media&token=2ae2aaca-a140-4574-b7b1-1b7080e6d8f7


KeyboardInterrupt: 

In [61]:
print(row)

(2, 2232639, '', 0, 'M', 'https://firebasestorage.googleapis.com/v0/b/kkalong-b4cec.appspot.com/o/casual%2Fcasual_1.png?alt=media&token=2ae2aaca-a140-4574-b7b1-1b7080e6d8f7', 5, None, 0, 512, 0, 'https://www.musinsa.com/app/goods/2232639/0', 0, 1, 'LAFUDGESTORE', 'casual', '0')
